# Use Case 6: Comparing mutated phosphosites

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
import CPTAC

Processing c:\users\david\documents\github\paynelab\cptac
  Running setup.py bdist_wheel for CPTAC: started
  Running setup.py bdist_wheel for CPTAC: finished with status 'done'
  Stored in directory: C:\Users\David\AppData\Local\Temp\pip-ephem-wheel-cache-xp_gu2no\wheels\c6\9e\67\70ffb2e65dc7dce8af1e18bcfde28338f7de6e805792af5ac2
Successfully built CPTAC
  Found existing installation: CPTAC 0.2
    Uninstalling CPTAC-0.2:
      Successfully uninstalled CPTAC-0.2
Loading Clinical Data...
Loading Proteomics Data...
Loading Transcriptomics Data...
Loading CNA Data...
Loading Phosphoproteomics Data...
Loading Somatic Data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until July 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter CPTAC.embargo() to open 

In [ ]:
sites = CPTAC.compare_phosphosites("ARID1A")

In [ ]:
mutations = CPTAC.get_somatic()["ARID1A"]
sites.insert(0, "Mutation", mutations)

In [ ]:
sites